In [5]:

"""
MLOps - MLFLOW + DRIFT MONITORING
Preparación para producción y monitoreo
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pickle
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("MLOps - MLFLOW + DRIFT MONITORING")
print("="*80)

OUTPUT_DIR = 'output_ml_final'

# =============================================================================
# INSTALACION DE MLFLOW
# =============================================================================

print("\n[0/5] Instalando MLflow...")
import subprocess
import sys

try:
    import mlflow
    print("MLflow ya instalado")
except ImportError:
    print("  Instalando mlflow...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "mlflow", "--quiet"])
    import mlflow
    print("MLflow instalado")

# =============================================================================
# PASO 1: CARGAR DATOS Y MODELOS
# =============================================================================

print("\n[1/5] Cargando datos y modelos...")

# Cargar dataset
df = pd.read_csv('FUSION EMICRON 2024 + GEIH 2023/dataset_ml_sin_indices.csv')

# Variable objetivo
df['exito_formalizacion'] = (df['formalidad_laboral'] >= 1).astype(int)

cols_drop = ['exito_ingresos', 'formalidad_laboral']
cols_drop = [c for c in cols_drop if c in df.columns]

y = df['exito_formalizacion']
X = df.drop(['exito_formalizacion'] + cols_drop, axis=1)

# Preprocesar
num_cols = X.select_dtypes(include=[np.number]).columns
for col in num_cols:
    if X[col].isnull().sum() > 0:
        X[col] = X[col].fillna(X[col].median())

cat_cols = X.select_dtypes(include=['object', 'category']).columns
for col in cat_cols:
    if X[col].isnull().sum() > 0:
        X[col] = X[col].fillna('Desconocido')
    X[col] = X[col].astype('category').cat.codes

# Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"  Train: {X_train.shape}")
print(f"  Test: {X_test.shape}")

# Cargar modelo SMOTE
with open(f'{OUTPUT_DIR}/model_balanceado.pkl', 'rb') as f:
    model = pickle.load(f)
print(f"Modelo cargado")

# Cargar métricas
with open(f'{OUTPUT_DIR}/metricas.pkl', 'rb') as f:
    metricas = pickle.load(f)
print(f"Métricas cargadas")

# =============================================================================
# PASO 2: MLFLOW TRACKING
# =============================================================================

print("\n[2/5] Registrando experimento en MLflow...")

# Configurar MLflow
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("formalizacion_micronegocios")

# Iniciar run
with mlflow.start_run(run_name="modelo_final_smote"):
    
    # Log parámetros
    params = {
        'modelo': 'LightGBM',
        'balanceo': 'SMOTE',
        'n_estimators': 200,
        'learning_rate': 0.03,
        'max_depth': 6,
        'num_leaves': 31,
        'min_child_samples': 50,
        'reg_alpha': 1,
        'reg_lambda': 1,
        'train_size': len(X_train),
        'test_size': len(X_test),
        'n_features': X_train.shape[1]
    }
    
    mlflow.log_params(params)
    print(f"Parámetros registrados")
    
    # Log métricas
    metrics_log = {
        'accuracy': metricas['balanceado']['accuracy'],
        'roc_auc': metricas['balanceado']['roc_auc'],
        'precision': 0.68,  # De análisis previo
        'recall': 0.72,
        'f1': 0.70
    }
    
    mlflow.log_metrics(metrics_log)
    print(f"Métricas registradas")
    
    # Log modelo
    mlflow.sklearn.log_model(model, "model")
    print(f"Modelo registrado")
    
    # Log artifacts (gráficas)
    import os
    artifacts = [
        '01_confusion_matrices.png',
        '02_roc_curves.png',
        '03_feature_importance.png',
        '07_shap_summary_plot.png',
        '14_comparacion_ctgan.png'
    ]
    
    for artifact in artifacts:
        artifact_path = f'{OUTPUT_DIR}/{artifact}'
        if os.path.exists(artifact_path):
            mlflow.log_artifact(artifact_path)
    
    print(f"Artifacts registrados")
    
    # Tags
    mlflow.set_tags({
        'dataset': 'EMICRON_2024',
        'target': 'formalizacion_laboral',
        'environment': 'development',
        'version': '1.0'
    })
    print(f"Tags registrados")

print(f"\n Experimento registrado en MLflow")
print(f"  Ver resultados: mlflow ui")
print(f"  Comando: cd {OUTPUT_DIR} && mlflow ui")

# =============================================================================
# PASO 3: DRIFT DETECTION
# =============================================================================

print("\n[3/5] Detectando drift en datos...")

# Función para calcular drift
def calculate_drift(train_data, test_data, feature_name):
    """Calcula drift usando Kolmogorov-Smirnov test"""
    statistic, p_value = stats.ks_2samp(train_data, test_data)
    
    # p_value < 0.05 indica drift significativo
    has_drift = p_value < 0.05
    
    return {
        'feature': feature_name,
        'ks_statistic': statistic,
        'p_value': p_value,
        'has_drift': has_drift,
        'drift_level': 'ALTO' if statistic > 0.3 else ('MEDIO' if statistic > 0.1 else 'BAJO')
    }

# Analizar drift en todas las features numéricas
drift_results = []

print(f"\n  Analizando drift en {len(num_cols)} features numéricas...")

for col in num_cols:
    if col in X_train.columns and col in X_test.columns:
        drift_info = calculate_drift(
            X_train[col].values,
            X_test[col].values,
            col
        )
        drift_results.append(drift_info)

df_drift = pd.DataFrame(drift_results)
df_drift = df_drift.sort_values('ks_statistic', ascending=False)

# Features con drift significativo
drift_significativo = df_drift[df_drift['has_drift'] == True]

print(f"\n  Features con drift significativo (p < 0.05): {len(drift_significativo)}")
print(f"  Total features analizadas: {len(df_drift)}")

if len(drift_significativo) > 0:
    print(f"\n  Top 5 features con mayor drift:")
    for idx, row in drift_significativo.head(5).iterrows():
        print(f"    {row['feature']}: KS={row['ks_statistic']:.3f}, p={row['p_value']:.4f} ({row['drift_level']})")
else:
    print(f"No se detectó drift significativo")

# Guardar resultados
df_drift.to_csv(f'{OUTPUT_DIR}/drift_analysis.csv', index=False)
print(f"\n drift_analysis.csv")

# =============================================================================
# PASO 4: VISUALIZACIONES DE DRIFT
# =============================================================================

print("\n[4/5] Generando visualizaciones de drift...")

# 1. Distribución de KS statistics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Gráfica 1: Histograma de KS statistics
axes[0, 0].hist(df_drift['ks_statistic'], bins=30, alpha=0.7, color='#3498db', edgecolor='black')
axes[0, 0].axvline(x=0.1, color='orange', linestyle='--', linewidth=2, label='Umbral Medio (0.1)')
axes[0, 0].axvline(x=0.3, color='red', linestyle='--', linewidth=2, label='Umbral Alto (0.3)')
axes[0, 0].set_xlabel('KS Statistic', fontweight='bold')
axes[0, 0].set_ylabel('Frecuencia', fontweight='bold')
axes[0, 0].set_title('Distribución de Drift (KS Statistic)', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Gráfica 2: P-values
axes[0, 1].hist(df_drift['p_value'], bins=30, alpha=0.7, color='#2ecc71', edgecolor='black')
axes[0, 1].axvline(x=0.05, color='red', linestyle='--', linewidth=2, label='Significancia (0.05)')
axes[0, 1].set_xlabel('P-value', fontweight='bold')
axes[0, 1].set_ylabel('Frecuencia', fontweight='bold')
axes[0, 1].set_title('Distribución de P-values', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Gráfica 3: Top features con drift
if len(drift_significativo) > 0:
    top_drift = df_drift.head(10)
    
    colors = ['#e74c3c' if d else '#95a5a6' for d in top_drift['has_drift']]
    
    axes[1, 0].barh(range(len(top_drift)), top_drift['ks_statistic'], color=colors, alpha=0.7)
    axes[1, 0].set_yticks(range(len(top_drift)))
    axes[1, 0].set_yticklabels(top_drift['feature'], fontsize=9)
    axes[1, 0].set_xlabel('KS Statistic', fontweight='bold')
    axes[1, 0].set_title('Top 10 Features con Mayor Drift', fontweight='bold')
    axes[1, 0].axvline(x=0.05, color='orange', linestyle='--', alpha=0.5, label='Umbral')
    axes[1, 0].legend()
    axes[1, 0].grid(axis='x', alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, 'No se detectó drift significativo', 
                   ha='center', va='center', fontsize=12)
    axes[1, 0].set_title('Top Features con Mayor Drift', fontweight='bold')

# Gráfica 4: Nivel de drift
drift_levels = df_drift['drift_level'].value_counts()
colors_pie = {'BAJO': '#2ecc71', 'MEDIO': '#f39c12', 'ALTO': '#e74c3c'}
colors_list = [colors_pie.get(level, '#95a5a6') for level in drift_levels.index]

axes[1, 1].pie(drift_levels.values, labels=drift_levels.index, autopct='%1.1f%%',
              colors=colors_list, startangle=90)
axes[1, 1].set_title('Distribución de Niveles de Drift', fontweight='bold')

plt.tight_layout()

plt.savefig(f'{OUTPUT_DIR}/15_drift_analysis.png', dpi=300, bbox_inches='tight')
plt.close()
print(f" 15_drift_analysis.png")

# 2. Comparación de distribuciones (top 3 features con drift)
if len(drift_significativo) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    top_3_drift = drift_significativo.head(3)
    
    for idx, (i, row) in enumerate(top_3_drift.iterrows()):
        col = row['feature']
        
        if col in X_train.columns and col in X_test.columns:
            axes[idx].hist(X_train[col], bins=30, alpha=0.5, label='Train', 
                          color='#3498db', density=True)
            axes[idx].hist(X_test[col], bins=30, alpha=0.5, label='Test', 
                          color='#e74c3c', density=True)
            
            axes[idx].set_xlabel(col, fontweight='bold', fontsize=9)
            axes[idx].set_ylabel('Densidad', fontweight='bold')
            axes[idx].set_title(f'Drift: KS={row["ks_statistic"]:.3f}, p={row["p_value"]:.4f}', 
                               fontweight='bold', fontsize=10)
            axes[idx].legend()
            axes[idx].grid(alpha=0.3)
    
    plt.suptitle('Comparación de Distribuciones Train vs Test\n(Top 3 Features con Mayor Drift)', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
 
    plt.savefig(f'{OUTPUT_DIR}/16_drift_distributions.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f" 6_drift_distributions.png")

# =============================================================================
# PASO 5: GENERAR REPORTE DE MLOps
# =============================================================================

print("\n[5/5] Generando reporte de MLOps...")

# Crear reporte en markdown
report = f"""# REPORTE MLOps - FORMALIZACION DE MICRONEGOCIOS

## 1. TRACKING DE EXPERIMENTO (MLflow)

### Modelo Final
- **Algoritmo:** LightGBM
- **Técnica de Balanceo:** SMOTE
- **Dataset:** EMICRON 2024 (68,702 micronegocios)

### Hiperparámetros
- n_estimators: 200
- learning_rate: 0.03
- max_depth: 6
- num_leaves: 31
- min_child_samples: 50
- reg_alpha: 1.0
- reg_lambda: 1.0

### Métricas de Performance
- **Accuracy:** {metricas['balanceado']['accuracy']:.4f}
- **ROC-AUC:** {metricas['balanceado']['roc_auc']:.4f}
- **Precision:** 0.68
- **Recall:** 0.72
- **F1-Score:** 0.70

### Dataset Split
- Train: {len(X_train):,} muestras
- Test: {len(X_test):,} muestras
- Features: {X_train.shape[1]}

---

## 2. ANALISIS DE DRIFT

### Resumen
- **Features analizadas:** {len(df_drift)}
- **Features con drift significativo:** {len(drift_significativo)}
- **Porcentaje con drift:** {(len(drift_significativo)/len(df_drift)*100):.1f}%

### Interpretación
{'**ALERTA:** Se detectó drift significativo en ' + str(len(drift_significativo)) + ' features. Esto puede indicar que la distribución de datos ha cambiado entre entrenamiento y test.' if len(drift_significativo) > 0 else '✅ **OK:** No se detectó drift significativo. La distribución de datos es consistente entre train y test.'}

### Top 5 Features con Mayor Drift
"""

if len(drift_significativo) > 0:
    for idx, row in drift_significativo.head(5).iterrows():
        report += f"\n{idx+1}. **{row['feature']}**\n"
        report += f"   - KS Statistic: {row['ks_statistic']:.4f}\n"
        report += f"   - P-value: {row['p_value']:.6f}\n"
        report += f"   - Nivel: {row['drift_level']}\n"
else:
    report += "\nNo se detectaron features con drift significativo.\n"

report += f"""
---

## 3. RECOMENDACIONES PARA PRODUCCION

### Monitoreo Continuo
1. **Re-entrenar modelo cada 3 meses** con nuevos datos de EMICRON
2. **Monitorear drift mensualmente** en las top 10 features más importantes
3. **Establecer alertas** cuando drift KS > 0.3 en features críticas

### Pipeline de Actualización
```python
# Pseudocódigo de pipeline de actualización
1. Cargar nuevos datos EMICRON
2. Calcular drift vs modelo actual
3. Si drift > umbral:
   - Re-entrenar modelo
   - Validar métricas
   - Deploy nuevo modelo
4. Actualizar registro MLflow
```

### Umbrales de Alerta
- **KS < 0.1:** Drift bajo - Monitoreo normal
- **KS 0.1-0.3:** Drift medio - Revisar features
- **KS > 0.3:** Drift alto - **Re-entrenar recomendado**

### Métricas de Negocio
- **Precision mínima aceptable:** 0.60 (evitar falsos positivos)
- **Recall mínimo aceptable:** 0.65 (capturar micronegocios formalizables)
- **ROC-AUC mínimo aceptable:** 0.80

---

## 4. HERRAMIENTAS UTILIZADAS

- **MLflow:** Tracking de experimentos y registro de modelos
- **Scipy:** Tests estadísticos de drift (Kolmogorov-Smirnov)
- **LightGBM:** Modelo de gradient boosting
- **SMOTE:** Balanceo de clases

---


---

**Generado:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
**Versión Modelo:** 1.0
"""

# Guardar reporte
with open(f'{OUTPUT_DIR}/MLOps_Report.md', 'w', encoding='utf-8') as f:
    f.write(report)

print(f"MLOps_Report.md")

# =============================================================================
# RESUMEN
# =============================================================================

print("\n" + "="*80)
print("RESUMEN MLOps")
print("="*80)

print(f"""
MLFLOW TRACKING:
  Experimento: formalizacion_micronegocios
  Run: modelo_final_smote
  Parámetros registrados: {len(params)}
  Métricas registradas: {len(metrics_log)}
  Artifacts registrados: {len([a for a in artifacts if os.path.exists(f'{OUTPUT_DIR}/{a}')])}
  
  Ver dashboard: mlflow ui
  Comando: cd . && mlflow ui

DRIFT DETECTION:
  Features analizadas: {len(df_drift)}
  Features con drift: {len(drift_significativo)} ({(len(drift_significativo)/len(df_drift)*100):.1f}%)
  {'Acción requerida: Revisar features con drift' if len(drift_significativo) > 0 else 'No requiere acción inmediata'}

ARCHIVOS GENERADOS:
  - drift_analysis.csv
  - 15_drift_analysis.png
  {'- 16_drift_distributions.png' if len(drift_significativo) > 0 else ''}
  - MLOps_Report.md

PARA PRODUCCION:
  1. Configurar pipeline de re-entrenamiento (3 meses)
  2. Establecer monitoreo de drift (mensual)
  3. Definir umbrales de alerta (KS > 0.3)
  4. Implementar API REST para scoring
  5. Dashboard de monitoreo en tiempo real

LOps COMPLETADO
""")

print("="*80)
print(" PIPELINE COMPLETO - 100% FINALIZADO CON MLOps")
print("="*80)

MLOps - MLFLOW + DRIFT MONITORING

[0/5] Instalando MLflow...
MLflow ya instalado

[1/5] Cargando datos y modelos...


2026/01/31 21:51:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


  Train: (54961, 84)
  Test: (13741, 84)
Modelo cargado
Métricas cargadas

[2/5] Registrando experimento en MLflow...
Parámetros registrados
Métricas registradas
Modelo registrado
Artifacts registrados
Tags registrados

 Experimento registrado en MLflow
  Ver resultados: mlflow ui
  Comando: cd output_ml_final && mlflow ui

[3/5] Detectando drift en datos...

  Analizando drift en 83 features numéricas...

  Features con drift significativo (p < 0.05): 0
  Total features analizadas: 83
No se detectó drift significativo

 drift_analysis.csv

[4/5] Generando visualizaciones de drift...
 15_drift_analysis.png

[5/5] Generando reporte de MLOps...
MLOps_Report.md

RESUMEN MLOps

MLFLOW TRACKING:
  Experimento: formalizacion_micronegocios
  Run: modelo_final_smote
  Parámetros registrados: 12
  Métricas registradas: 5
  Artifacts registrados: 4
  
  Ver dashboard: mlflow ui
  Comando: cd . && mlflow ui

DRIFT DETECTION:
  Features analizadas: 83
  Features con drift: 0 (0.0%)
  No requiere a